# Create Drift Monitors Using the Arize GraphQL API

### Step 1: Initialize the GraphQL Client using your developer API Key

In [ ]:
!pip install gql[all]
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

### Get your API key
First - make sure you have developer permissions. If you are able to visit the [API explorer](https://app.arize.com/graphql), you have developer permissions, if not, please ask your Account Admin to provide you with access. 

The API key can be retrieved from the [API explorer](https://app.arize.com/graphql) page. Click the button on the top right called "Get Your API Key." A modal will pop up with your key, copy that into the `API_KEY` constant below. 

NOTE: this key is different than the SDK key used to send data to Arize. 

In [ ]:
API_KEY = "API_KEY"

# Select your transport with a defined url endpoint
transport = RequestsHTTPTransport(
    url="https://app.arize.com/graphql/", headers={"x-api-key": API_KEY}
)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

### Step 2: Execute a GraphQL query to get all your models

In [ ]:
# We start this query from your space. Spaces have globally unique IDs. You can get your space ID by visiting app.arize.com.
# The url will be in this format: https://app.arize.com/organizations/:orgId/spaces/:spaceId
# NOTE: this is not the same as the space key used to send data using the SDK
SPACE_ID = "SPACE_ID"


# A re-usable query for fetching your models, a page at a time
models_query = gql(
    """
    query getModels($spaceId: ID!, $cursor: String) {
        space: node(id: $spaceId) {
            ... on Space {
                name
                models (first: 50, after: $cursor) {
                    pageInfo {
                        endCursor
                    }
                    edges {
                        model: node {
                            id
                            name
                        }
                    }
                }
            }
        }
    }
"""
)

# Base query parameters for fetching models
params = {"spaceId": SPACE_ID}
# An array of models that we will append to
models = []
space_name = ""


# Execute the query on the transport. Continue to pull data until there is no more monitors
while True:
    paged_response = client.execute(models_query, params)
    space_name = paged_response["space"]["name"]
    # Append the monitors to your list
    models.extend(paged_response["space"]["models"]["edges"])
    # If there is another page of information, point the cursor to the next page and fetch more
    end_cursor = paged_response["space"]["models"]["pageInfo"]["endCursor"]
    print("pageInfo end_cursor %s" % (end_cursor))
    if end_cursor:
        print("There is another page of models. Loading more.")
        params["cursor"] = end_cursor
    else:
        # No more models to pull. The list is complete!
        break

print("Retrieved {} models".format(len(models)))

#### Step 2a: Print our some of the models to check that exports are expected

In [ ]:
import pandas as pd

# The models have a nested JSON structure, let's flatten it into a data frame
models_df = pd.json_normalize(models, sep=".")
models_df.head()

### Important: Step 3 involves exporting, manipulating, and importing modified data leveraging google sheets (recommended). If you prefer to use a standard .csv instead, consult the following cell. Otherwise, ignore and proceed to step 3.

In [ ]:
# only consult this section if using csv, then skip ahead to step 4.
# uncomment the following code to create a monitors.csv file in your current directory,
# modify the csv by adding column headers and rows (consult monitor_columns in step 3a.),
# and convert back to a pandas dataframe

# monitors_csv = monitors_df.to_csv('monitors.csv')
# dq_monitors_df = pd.read_csv('monitors.csv')
# dq_monitors_df.head()

### Step 3: Save models in a spreadsheet to reference for modelId

In [ ]:
!pip install gspread
# src: https://colab.research.google.com/notebooks/snippets/sheets.ipynb
from google.colab import auth

auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()

gc = gspread.authorize(creds)

sheet_name = f"{space_name} models"
sh = gc.create(sheet_name)

# Open our new sheet and add some data.
worksheet = gc.open(sheet_name).sheet1

# Let's now write the dataframe to google sheets
worksheet.update([models_df.columns.values.tolist()] + models_df.values.tolist())

# print the URL
print(sh.url)

#### 3a. Create a second sheet within the spreadsheet to populate with monitor inputs

We want to create a second sheet for creating performance monitors. Reference the first sheet to populate the modelId column for the creation of performance monitors

In [ ]:
# consult the documentation for examples about these fields and their optionality
# optional fields may be left blank in the sheet
monitor_columns = [
    "name",
    "modelId",
    "driftMetric",
    "operator",
    "dimensionName",
    "dimensionCategory",
    "evaluationWindowLengthSeconds",
    "threshold",
    "filters",  # set to 1 in cell if there is a filter
    "filters.dimensionType",
    "filters.operator",
    "filters.name",
    "filters.values",
    "baseline",
    "baseline.useModelPrimaryBaseline" "contacts",
]

In [ ]:
# run this cell only once to create a new worksheet and populate it with column headers
create_worksheet_name = "example_inputs"
sh.add_worksheet(title=create_worksheet_name, rows="100", cols="20")
worksheet = sh.worksheet(create_worksheet_name)
worksheet.update([monitor_columns])

#### Step 3b. Import the data back into a dataframe

In [ ]:
worksheet = sh.worksheet(create_worksheet_name)

# convert the spreadsheet rows back into a pandas dataframe
drift_monitors_df = pd.DataFrame(worksheet.get_all_records())
drift_monitors_df.head()

### Step 4: Create drift monitors

Create a drift monitor for each row in the sheet. The below example is specifically for creating drift monitors using the model primary baseline.

In [ ]:
create_drift_monitor_mutation = gql(
    """
    mutation createDriftMonitorMutation(
      $name: String!, 
      $modelId: ID!, 
      $dimensionName: String,
      $dimensionCategory: DimensionCategory!,
      $driftMetric: DriftMetric!, 
      $evaluationWindowLengthSeconds: Float, 
      $threshold: Float, 
      $operator: ComparisonOperator!, 
      $filters: [DimensionFilterInput!],
      $baseline: DriftMonitorBaseline 
      $contacts: [MonitorContactInput!]
    ) {
      createDriftMonitor(
        input: { 
          name: $name, 
          modelId: $modelId, 
          dimensionName: $dimensionName,
          dimensionCategory: $dimensionCategory,
          driftMetric: $driftMetric, 
          evaluationWindowLengthSeconds: $evaluationWindowLengthSeconds, 
          threshold: $threshold, 
          operator: $operator, 
          filters: $filters, 
          baseline: $baseline,
          contacts: $contacts
        }
      ) {
        monitor { id, name, threshold }
      }
    }
"""
)

n = 0

for _, row in drift_monitors_df.iterrows():

    seven_days = 259200  # seconds

    param_keys = [
        "name",
        "modelId",
        "dimensionName",
        "dimensionCategory",
        "evaluationWindowLengthSeconds",
        "operator",
        "driftMetric",
    ]

    params = {key: row[key] for key in param_keys if key in row}

    baseline_params = {}
    contact_params = {}

    if row["evaluationWindowLengthSeconds"]:
        params["evaluationWindowLengthSeconds"] = row["evaluationWindowLengthSeconds"]
    else:
        params["evaluationWindowLengthSeconds"] = seven_days
    if row["baseline"]:
        baseline_params["useModelPrimaryBaseline"] = row["baseline.useModelPrimaryBaseline"]
    if row["contacts"]:
        contact_params["notificationChannelType"] = "email"
        contact_params["emailAddress"] = row["contacts"]
        params["contacts"] = contact_params

    result = client.execute(create_drift_monitor_mutation, params)
    n += 1
    print(f"{n} created data quality monitor with {params}")